# Efficency scanner

### This python script can record arbitrary data from a serial connection. For a set time even. Please note that the outputted data is likely formatted in mac line endings (because that is what Arduinos send). So if you want to use the data in another program, especially the analysis program, conversion to Linux line endings must be done. On Windows Notepad++ can do this.


Note: the maximum output speed of the DAQ-card is about 30 events per second 

In [2]:
import serial
import time

In [3]:
# settings

recording_secs_per_setting = 60*2

start_threshold = 5
stop_threshold = 180
step_size = 5

detector_mode = "2E" # Coincidence (threefold) and Channels (1,2,3), no veto
#detector_mode = "2E" # Coincidence (threefold) and Channels (1,2,3), no veto

output_file = time.strftime("%Y%m%d-%H%M")+"_Efficency_scan"
output_file += "_start-"+str(start_threshold)+"_stop-"+str(stop_threshold)+"_step-"+str(step_size)
output_file += "_sec_per_rec_"+str(recording_secs_per_setting)
output_file += "_detector_mode_"+str(detector_mode)+".txt"
print("Output file name: " + output_file)
serial_port = 'COM14'
baud_rate = 115200

Output file name: 20171113-1049_Efficency_scan_start-5_stop-180_step-5_sec_per_rec_120_detector_mode_2E.txt


In [4]:
# start the serial connection
ser = serial.Serial(serial_port, baud_rate, timeout=0.5)

In [5]:
# Functions

def turn_off_auto_reset():
    commands = "ST 0\r"
    setting_successfull = False
    while setting_successfull == False:
        ser.write(commands)
        # check if everything was recieved as prompted
        curr_line = str(ser.readline())
        curr_line = str(ser.readline())
        checking_line = curr_line
        if "ST Disabled" in checking_line:
            setting_successfull = True
            print("Disabling of the auto reset was sucessfull")
            break
        else:
            print("Disabling of the auto reset mode failed! Retring now.")
            setting_successfull = False
            continue

def set_detector_mode(detector_mode):
    commands = "WC 00 "+detector_mode+"\r"
    setting_successfull = False
    while setting_successfull == False:
        ser.write(commands)
        # check if everything was recieved as prompted
        curr_line = str(ser.readline())
        curr_line = str(ser.readline())
        checking_line = curr_line
        returned_parameters = checking_line.split("=")
        result_setting = returned_parameters[1].replace('\r\n', '')
        if (result_setting == detector_mode):
            setting_successfull = True
            print("Setting of the coincidence mode was sucessfull")
            break
        else:
            print("Setting of the coincidence mode failed! Retring now.")
            setting_successfull = False
            continue
    

def set_thresh(threshs):
    setting_successfull = False
    while setting_successfull == False:
        commands = "TL 0 "+str(threshs[0])+"\rTL 1 "+str(threshs[1])+"\rTL 2 "+str(threshs[2])+"\rTL 3 "+str(threshs[3])+"\r"
        ser.write(commands)
        # poll until all commands were recieved
        curr_line = ser.readline()
        while not (curr_line == ''):
            curr_line = ser.readline()
        # check if everything was recieved as prompted
        ser.write("TL\r")
        curr_line = str(ser.readline())
        curr_line = str(ser.readline())
        checking_line = curr_line
        checking_line = checking_line.replace("L0","").replace("L1","").replace("L2","").replace("L3","")
        returned_parameters = checking_line.split("=")
        for i in range(len(threshs)):
            if(int(returned_parameters[i+1]) != int(threshs[i])):
                print("Setting of thresholds failed! Retring now.")
                setting_successfull = False
                continue
        setting_successfull = True
        print("Setting of thresholds was sucessfull. Set to: "+str(threshs))
        break
        
def stop_detector_output():
    # stop the output from the detector and poll last lines
    print("Stopping detector output")
    ser.write("CD\r")
    curr_line = ser.readline()
    while not (curr_line == ''):
        curr_line = ser.readline()
    print("Detector output stopped")

In [6]:
turn_off_auto_reset()
stop_detector_output()
# setup the detector
set_thresh([start_threshold for i in range(4)])
set_detector_mode(detector_mode)

Stopping detector output
Detector output stopped
Setting of thresholds was sucessfull. Set to: [5, 5, 5, 5]
Setting of the coincidence mode was sucessfull


In [ ]:
current_thresh = start_threshold
# write into output file
with open(output_file, 'w') as f:
    f.write("recording_secs_per_setting: "+str(recording_secs_per_setting)+"\r\r")
    while current_thresh <= stop_threshold:
        # set new thresholds
        set_thresh([current_thresh for i in range(4)])
        f.write("Rien\r\r")                # Make sure we have some space
        f.write("Rien\r\r")
        f.write("Rien\r\r")
        f.write("Current Threshold: "+str(current_thresh)+"\r\r")
        # start detector
        ser.write("CE\r")
        # start the timer 
        start_time = time.time()
        # read lines until we are supposed to stop
        while time.time() < (start_time+recording_secs_per_setting):
            line = ser.readline()   # read a '\n' terminated line
            # remove the \n at the end
            line = line.split('\n')[0]
            f.write(line)
        # stop the detector
        stop_detector_output()
        
        # read the counters
        ser.write("DS\r")
        # write lines into file:
        curr_line = ser.readline()
        while not (curr_line == ''):
            # remove the \n at the end
            curr_line = curr_line.split('\n')[0]
            f.write(curr_line)
            curr_line = ser.readline()
        f.flush()
        
        # increment the threshold
        current_thresh += step_size
        
        steps_left = (stop_threshold-current_thresh+step_size)/step_size
        time_left = recording_secs_per_setting * steps_left
        print("Estimated leftover runtime: {:.1f} [min]".format(time_left/60.0))

Setting of thresholds was sucessfull. Set to: [5, 5, 5, 5]
